In [1]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os 
import tensorflow as tf
import random
import keras
from keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

2024-01-25 21:06:27.051191: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-25 21:06:27.726756: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_csv_filename = '../merged_feret_train.csv' 
test_csv_filename = '../merged_feret_test.csv'
train_df = pd.read_csv(train_csv_filename)
test_df = pd.read_csv(test_csv_filename)

In [3]:
train_df = train_df.sample(frac = 1)
test_df = test_df.sample(frac = 1)

In [5]:
generated_df = pd.read_csv("../feret_generated.csv")
train_df = pd.concat([train_df, generated_df], ignore_index=True)

In [5]:
root_path = "../colored/"
generation_path = "../accepted/"
train_df['image_path'] = train_df['filename'].apply(lambda x: os.path.join(root_path if "final" in x else generation_path, x))
test_df['image_path'] = test_df['filename'].apply(lambda x: os.path.join(root_path, x))

In [6]:
race_four_test = test_df[test_df['race'] == 4]
race_three_test = test_df[test_df['race'] == 3]
race_two_test = test_df[test_df['race'] == 2]
race_one_test = test_df[test_df['race'] == 1]
race_zero_test = test_df[test_df['race'] == 0]

chosen_race = test_df

In [8]:
train_image_paths = train_df['image_path'].tolist()
test_image_paths = chosen_race['image_path'].tolist()

In [10]:
# Read and resize images
def preprocess_image(file_path, target_size=(512, 768)):
    img = cv2.imread(file_path) 
    img = cv2.resize(img, (512,512))
    x, y = 85, 0
    crop_img = img[y:y+512, x:x+342]
    resized_img = cv2.resize(crop_img, target_size)
    final_img = resized_img.astype("float") / 255.0  # Normalize pixel values
    # cv2.imshow("test", resized_img)
    # cv2.waitKey(0)
    return final_img

In [11]:
# Apply preprocessing to the entire dataset
X_train = [preprocess_image(file_path) for file_path in train_image_paths]
X_test = [preprocess_image(file_path) for file_path in test_image_paths]

X_train = np.array(X_train)
X_test = np.array(X_test)

In [12]:
X_test.shape

(38, 768, 512, 3)

In [13]:
train_gender_labels = train_df['gender'].tolist()
test_gender_labels = test_df['gender'].tolist()
train_race_labels = train_df['race'].tolist()
test_race_labels = chosen_race['race'].tolist()

In [14]:
from sklearn.preprocessing import LabelEncoder

# Create label encoders for gender and race
gender_encoder = LabelEncoder()
race_encoder = LabelEncoder()

# Apply label encoding to gender
y_train_gender_encoded = gender_encoder.fit_transform(train_gender_labels)
y_test_gender_encoded = gender_encoder.transform(test_gender_labels)

# Apply label encoding to race
y_train_race_encoded = race_encoder.fit_transform(train_race_labels)
y_test_race_encoded = race_encoder.transform(test_race_labels)


In [16]:
input_shape = (768,512, 3)
num_classes_race = 5 

# race_model = Sequential()
# race_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
# race_model.add(MaxPooling2D((2, 2)))
# race_model.add(Flatten())
# race_model.add(Dense(128, activation='relu'))
# race_model.add(Dense(num_classes_race, activation='softmax'))



input_layer = keras.Input(shape=input_shape, name="Input image")
x = layers.Conv2D(16, 3, activation="relu")(input_layer)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(64, 3, activation="relu")(x)
x = layers.Conv2D(64, 3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)

out_a = keras.layers.Dense(1, activation='sigmoid', name='g_clf')(x)
out_b = keras.layers.Dense(1, activation='linear', name='a_reg')(x)
 
model = keras.Model( inputs = input_layer, outputs = [out_a, out_b], name="age_gender_model")

race_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2024-01-25 17:27:43.222524: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [17]:
race_model.fit(X_train, y_train_race_encoded, epochs=5, batch_size=16, validation_data=(X_test, y_test_race_encoded))

Epoch 1/5


2024-01-25 17:27:56.474483: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4303355904 exceeds 10% of free system memory.


57/57 [==============================] - 100s 2s/step - loss: 60.4733 - accuracy: 0.5110 - val_loss: 4.0435 - val_accuracy: 0.0000e+00
Epoch 2/5
57/57 [==============================] - 100s 2s/step - loss: 0.9631 - accuracy: 0.6886 - val_loss: 4.4184 - val_accuracy: 0.0789
Epoch 3/5
57/57 [==============================] - 99s 2s/step - loss: 0.5835 - accuracy: 0.8015 - val_loss: 6.9892 - val_accuracy: 0.0789
Epoch 4/5
57/57 [==============================] - 100s 2s/step - loss: 0.3076 - accuracy: 0.9035 - val_loss: 5.6808 - val_accuracy: 0.1053
Epoch 5/5
57/57 [==============================] - 99s 2s/step - loss: 0.1191 - accuracy: 0.9770 - val_loss: 9.4427 - val_accuracy: 0.0526


In [18]:
loss, accuracy = race_model.evaluate(X_test, y_test_race_encoded)
print(f"Test Accuracy: {accuracy}")

2/2 [==============================] - 1s 127ms/step - loss: 9.4427 - accuracy: 0.0526
Test Accuracy: 0.05263157933950424


In [ ]:
from keras.layers import Input, Dense, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D, GlobalMaxPool2D, Dropout, Flatten
from keras.optimizers import SGD
from keras.models import Model
from keras.regularizers import l2
from keras.layers import Activation

input = Input(shape = (100,100,1))
conv1 = Conv2D(32,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(input)
conv1 = Dropout(0.1)(conv1)
conv1 = Activation('relu')(conv1)
pool1 = MaxPooling2D(pool_size = (2,2)) (conv1)
conv2 = Conv2D(64,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool1)
conv2 = Dropout(0.1)(conv2)
conv2 = Activation('relu')(conv2)
pool2 = MaxPooling2D(pool_size = (2,2)) (conv2)
conv3 = Conv2D(128,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool2)
conv3 = Dropout(0.1)(conv3)
conv3 = Activation('relu')(conv3)
pool3 = MaxPooling2D(pool_size = (2,2)) (conv3)
conv4 = Conv2D(256,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool3)
conv4 = Dropout(0.1)(conv4)
conv4 = Activation('relu')(conv4)
pool4 = MaxPooling2D(pool_size = (2,2)) (conv4)
flatten = Flatten()(pool4)
dense_1 = Dense(128,activation='relu')(flatten)
drop_1 = Dropout(0.2)(dense_1)

# for race prediction
_ = Dense(units=512, activation='relu')(bottleneck)
race_output = Dense(units=len(RACE_ID_MAP), activation='softmax', name='race_output')(_)

# Model compile
model = Model(inputs=input,outputs=output)
model.compile(optimizer="adam",loss=["sparse_categorical_crossentropy"],metrics=['accuracy'])

model_path='./output/gender_model.h5'
checkpointer = ModelCheckpoint(model_path, monitor='loss',verbose=1,save_best_only=True,
                               save_weights_only=False, mode='auto',save_freq='epoch')
callback_list=[checkpointer]